## Slot Feeling

In [2]:
import os
import pickle
import random
import shutil

from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
from datasets import load_metric, Dataset

from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer, pipeline
import transformers
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

/home/leffff/PycharmProjects/ml-talent-match/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SEED = 21

In [4]:
import re
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
        
    def __len__(self):
        return len(self.labels)

In [5]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False


seed_all(SEED)

In [6]:
import re

def remove_keys(input_string):
    pattern = r'\[(\w+)\s*:\s*([^\]]+)\]'
    cleaned_string = re.sub(pattern, r'\2', input_string)
    return cleaned_string

input_string = "разбуди меня в [time : пять утра] на [date : этой неделе]"
cleaned_string = remove_keys(input_string)
print(cleaned_string)

разбуди меня в пять утра на этой неделе


In [5]:
from datasets import load_dataset


_TAGS =['experienceItem',
 'city',
 'first_name',
 'languageItem',
 'country',
 'languageItems',
 'last_name',
 'uuid',
 'key_skills',
 'birth_date',
 'about',
 'educationItem'
]

In [6]:
import json

f = open('/content/case_2_reference_without_resume_sorted.json')


data = json.load(f)

In [10]:
_TAGS = ['experienceItem',
 'city',
 'first_name',
 'languageItem',
 'country',
 'languageItems',
 'last_name',
 'uuid',
 'key_skills',
 'birth_date',
 'about',
 'educationItem',
         'O']


In [7]:
def create_dataset_with_repeated_labels(data_dict):
    text = []
    labels = []

    for key, value in data_dict.items():
        value_parts = str(value).split()
        text.extend(value_parts)
        labels.extend([key] * len(value_parts))

    text_string = ' '.join(text)

    dataset = {
        "text": text_string,
        "labels": labels
    }

    return dataset


In [8]:
datasets = [create_dataset_with_repeated_labels(d) for d in data['resumes']]

In [9]:
import pandas as pd

def dicts_list_to_dataframe_adjusted(dicts_list):
    uuids = []
    texts = []
    labels_lists = []
    for d in dicts_list:
        text_elements = d['text'].split()
        labels = d['labels']
        uuids.append(text_elements[0])
        texts.append(' '.join(text_elements[1:]))  # Skip the UUID
        adjusted_labels = [label for label in labels[1:] if label != 'uuid']
        labels_lists.append(adjusted_labels)
    df_adjusted = pd.DataFrame({"uuid": uuids, "text": texts, "labels": labels_lists})
    return df_adjusted

def add_text_split_column(df):
    df['tokens'] = df['text'].apply(lambda x: x.split())
    return df


df_adjusted = dicts_list_to_dataframe_adjusted(datasets)
df_with_text_split = add_text_split_column(df_adjusted)

adjusted_csv_file_path = 'resumes_train.csv'
df_with_text_split.to_csv(adjusted_csv_file_path, index=False)

df_with_text_split.head(), adjusted_csv_file_path

(                                   uuid  \
 0  0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33   
 1  f8b69e24-e2c0-3186-9578-380835eb2ee7   
 2  e3976e74-e71b-34db-8e98-08dc422fa567   
 3  9a9c3ff1-49f8-30dd-a294-e56fc60cae64   
 4  6561771c-7ef3-3e50-ab3a-ba8547201480   
 
                                                 text  \
 0  Клим Тетерина 1991-05-23 Россия Москва Личные ...   
 1  Алиса Ситникова 1990-07-18 Россия Нижний Новго...   
 2  Розалина Андреев 1990-01-01 Россия Санкт-Петер...   
 3  Антон Кудрявцева 1990-01-01 Россия Москва Общи...   
 4  Александра Панова 1995-01-01 Россия Москва О с...   
 
                                               labels  \
 0  [first_name, last_name, birth_date, country, c...   
 1  [first_name, last_name, birth_date, country, c...   
 2  [first_name, last_name, birth_date, country, c...   
 3  [first_name, last_name, birth_date, country, c...   
 4  [first_name, last_name, birth_date, country, c...   
 
                                              

In [12]:
import pandas as pd

def add_ner_tags_column(df, label2id):
    df['ner_tags'] = df['labels'].apply(lambda labels: [label2id[label] for label in labels if label in label2id])
    return df



df_with_ner_tags = add_ner_tags_column(df_with_text_split, label2id)

df_with_ner_tags


,uuid,text,labels,tokens,ner_tags
0,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33,Клим Тетерина 1991-05-23 Россия Москва Личные ...,"[first_name, last_name, birth_date, country, c...","[Клим, Тетерина, 1991-05-23, Россия, Москва, Л...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 10, 10, 10, 10..."
1,f8b69e24-e2c0-3186-9578-380835eb2ee7,Алиса Ситникова 1990-07-18 Россия Нижний Новго...,"[first_name, last_name, birth_date, country, c...","[Алиса, Ситникова, 1990-07-18, Россия, Нижний,...","[2, 6, 9, 4, 1, 1, 10, 10, 10, 10, 10, 10, 10,..."
2,e3976e74-e71b-34db-8e98-08dc422fa567,Розалина Андреев 1990-01-01 Россия Санкт-Петер...,"[first_name, last_name, birth_date, country, c...","[Розалина, Андреев, 1990-01-01, Россия, Санкт-...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 8, 8, 8, 8, 8,..."
3,9a9c3ff1-49f8-30dd-a294-e56fc60cae64,Антон Кудрявцева 1990-01-01 Россия Москва Общи...,"[first_name, last_name, birth_date, country, c...","[Антон, Кудрявцева, 1990-01-01, Россия, Москва...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 10, 10, 10, 10..."
4,6561771c-7ef3-3e50-ab3a-ba8547201480,Александра Панова 1995-01-01 Россия Москва О с...,"[first_name, last_name, birth_date, country, c...","[Александра, Панова, 1995-01-01, Россия, Москв...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 10, 10, 10, 10..."
...,...,...,...,...,...
108,82df355a-235e-3046-9e6e-782ddf1600eb,Кристина Яковлева 1995-06-17 Россия Москва Non...,"[first_name, last_name, birth_date, country, c...","[Кристина, Яковлева, 1995-06-17, Россия, Москв...","[2, 6, 9, 4, 1, 10, 8, 0, 0, 0, 0, 0, 0, 0, 0,..."
109,915597ce-24e5-31fa-8dca-29437f49f839,Аполлон Белякова 1987-01-01 Россия Мураши Боле...,"[first_name, last_name, birth_date, country, c...","[Аполлон, Белякова, 1987-01-01, Россия, Мураши...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 10, 10, 10, 10..."
110,f288a532-0b58-30cb-ac3c-f87e53984719,Клавдия Пономарёв 1984-11-22 Россия Москва В с...,"[first_name, last_name, birth_date, country, c...","[Клавдия, Пономарёв, 1984-11-22, Россия, Москв...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 10, 10, 10, 10..."
111,3e3a379f-226e-305e-b7d8-cf341e00cbd7,Фёдор Харитонова 1993-06-20 Турция Анталия Пои...,"[first_name, last_name, birth_date, country, c...","[Фёдор, Харитонова, 1993-06-20, Турция, Антали...","[2, 6, 9, 4, 1, 10, 10, 10, 10, 10, 10, 10, 10..."


In [13]:
df_with_ner_tags = df_with_ner_tags.drop(columns=['uuid', 'text', 'labels'])

In [14]:
dataset = Dataset.from_pandas(df_with_ner_tags)

In [66]:
def listmerge1(lstlst):
    all=[]
    for lst in lstlst:
        for el in lst:
            all.append(el)
    return all

In [67]:
all_labels = df_with_ner_tags['ner_tags']
all_labels = listmerge1(all_labels)

In [15]:
dataset = dataset.train_test_split(0.2)

In [11]:
id2label = {}
label2id = {}
for i in range(len(_TAGS)):
    id2label[i]=_TAGS[i]
    label2id[_TAGS[i]]=i

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
example = dataset['train'][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

In [24]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [25]:
tokenized_train = dataset['train'].map(tokenize_and_align_labels, batched=True)
tokenized_test = dataset['test'].map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [27]:
import evaluate

seqeval = evaluate.load("seqeval")

In [28]:
import numpy as np


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [_TAGS[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [_TAGS[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [100]:
all_labels.append(7)
all_labels.append(12)

In [102]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

loss_weights = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)

In [139]:
from torch import nn


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to('cuda')
        
        outputs = model(**inputs)
        logits = outputs.get("logits").to('cuda')
        
        
        if labels is not None:
          weights = loss_weights
          loss_fct = torch.nn.CrossEntropyLoss(weight=torch.Tensor(weights)).to('cuda')
          loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
          return (loss, outputs) if return_outputs else loss

In [140]:
model = AutoModelForTokenClassification.from_pretrained(
    "cointegrated/LaBSE-en-ru", 
    num_labels=len(_TAGS), 
    id2label=id2label, 
    label2id=label2id
).to('cuda')

training_args = TrainingArguments(
    overwrite_output_dir=True,
    save_total_limit = 2,
    output_dir = '/content/model_v2',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    seed=SEED,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/LaBSE-en-ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.280269,0.123020,0.705882,0.209524,0.724669
2,No log,0.217499,0.245810,0.705882,0.364641,0.819080
3,No log,0.176174,0.712389,0.860963,0.779661,0.928501
4,No log,0.234224,0.639098,0.909091,0.750552,0.932254
5,No log,0.120469,0.694561,0.887701,0.779343,0.954967
6,No log,0.073010,0.709544,0.914439,0.799065,0.962670
7,No log,0.080770,0.766816,0.914439,0.834146,0.964053
8,No log,0.065413,0.745614,0.909091,0.819277,0.973731
9,No log,0.060706,0.814286,0.914439,0.861461,0.973929
10,No log,0.062707,0.806604,0.914439,0.857143,0.973336


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: first_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: last_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: birth_date seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: country seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: city seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequ

'\nclassifier = pipeline("ner", model=\'/kaggle/working/NER_model \'+str(i) + \'/checkpoint-432\')\nshutil.rmtree(path=\'/kaggle/working/NER_model \'+str(i))\n\nfor k in tqdm(range(len(X_folds_test_add_token[i]))):\n    tokenized_text_add_token = X_folds_test_add_token[i][k]\n    X_folds_test_add_token[i][k] = add_predicted_tokens(tokenized_text=tokenized_text_add_token, classifier=classifier)\n    tokenized_text_replace_token = X_folds_test_replace_token[i][k]\n    X_folds_test_replace_token[i][k] = replace_predicted_tokens(tokenized_text=tokenized_text_replace_token, classifier=classifier)\n\n\n\nprint(X_folds_test_replace_token[i][-1])\n'

In [142]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: first_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: last_name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: birth_date seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: country seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: city seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequ

{'eval_loss': 0.060705896466970444,
 'eval_precision': 0.8142857142857143,
 'eval_recall': 0.9144385026737968,
 'eval_f1': 0.8614609571788414,
 'eval_accuracy': 0.9739285008888011,
 'eval_runtime': 0.983,
 'eval_samples_per_second': 23.397,
 'eval_steps_per_second': 6.104,
 'epoch': 10.0}

In [146]:
import shutil
dir_name = '/content/model_v2/checkpoint-230'
output_filename = 'model_v2'
shutil.make_archive(output_filename, 'zip', dir_name)

'/content/model_v2.zip'

In [9]:
classifier = pipeline("ner", model='models/model', aggregation_strategy="simple")

In [13]:
text = """Пётр Игнатьев 1992-09-10 Россия Санкт-Петербург None Java, Java SE, Spring Framework, Spring Boot, PostgreSQL, Kafka, Apache Maven, OpenShift, Hibernate ORM, Liquibase, Jenkins, Bitbucket, CSS, HTML, JDBC, jUnit, Spring Test, Mockito, Git, GitLab, Jira, Confluence [{'starts': '2022-03-01', 'ends': None, 'employer': 'ООО Компания Метиз', 'city': 'Домодедово', 'position': 'Главный инженер по разработке', 'description': ' Примеры достижений: • Полностью сделана под ключ задача интеграции со сторонним сервисом, что позволит обеспечить процесс выплат, а также решит задачу построения аналитических данных, с помощью которых можно будет управлять процессом продаж; • Решена проблема загрузки большого объема данных, что решило проблему утечки памяти и увеличило производительность в 10 раз; • Реализована загрузка реестров. Обязанности: • Написание java кода; • Проведение code-review; • Участие в командных церемониях: планирование backlog, демо, ретро и т. д.; • Участие в процессах построения CI, CD, НТ, внедрения; • Проведение оценки трудоемкости задач; • Декомпозиция задач. Технологии используемые на проекте: • Java • Spring • Spring Boot • SQL (PostgreSQL) • jUnit, Spring Test, Mockito • Maven, Git, Bitbucket • Liquibase • Jenkins • Kafka • Docker '}, {'starts': '2018-02-01', 'ends': '2022-03-01', 'employer': 'ПАО МоторМетизТелеком', 'city': 'Зарайск', 'position': 'Java-разработчик', 'description': ' Примеры достижений: • Полностью переделан алгоритм валидации версий документации проектировщиков. Увеличена производительность в 8 раз и надежность. • Реализован планировщик задач проектировщиков Обязанности: Разработка нового функционала, доработка существующего, поиск и устранение багов в бэкенд части PLM системы, которая внедрена на проекте строительства АЭС Ханхикиви-1 в Финляндии; Командировки к заказчикам в финский офис компании RAOS Project Oy в городе Хельсинки; Обследование бизнес-процессов заказчика и составление технического задания; Разработка документации и инструкций для разрабатываемого ПО; Разработка интеграционных решений. Технологии используемые на проекте: • Java • Spring • Spring Boot • SQL (PostgreSQL) • jUnit, Spring Test, Mockito • Maven, Git, GitLab '}, {'starts': '2014-04-01', 'ends': '2018-01-01', 'employer': 'ЗАО ОрионЛифтВодНаладка', 'city': 'Зарайск', 'position': 'Ведущий инженер', 'description': ' Работа на зарубежную компанию в холдинг которой входит KFC и Pizza Hut • Ведение проектов по внедрению кассовых систем и прочих IT проектов; • Поддержка мобильных приложений компании; • Администрирование Active Directory; • Настройка кассовых систем IS, R-keeper, iiko; • Внедрение новых технических корпоративных решений; • Создание Web-отчетов на базе php, HTML, SQL; • Разработка инструкций, технических заданий и документации; • Анализ и выявление системных ошибок, поиск решений; • Коммуникация с иностранными отделами компании; • Контроль и корректировка действий сервисных компаний; • Администрирование операционных систем Windows 7, 8, 10 и пакета Microsoft OFFICE; • Первая линия поддержки всех типов пользователей; • Регистрация заявок пользователей. '}, {'starts': '2013-08-01', 'ends': '2014-04-01', 'employer': 'ПАО КазаньIT', 'city': 'Шаховская', 'position': 'Дежурный сотрудник технической поддержки', 'description': ' • Поддержка пользователей хостинга по телефону, почте, тикету; • Настройка серверов хостинга под задачи пользователя, настройка php.ini, восстановление сайтов, помощь в работе ПУ аккаунта, редактирование .htaccess; • Решение задач, связанных с хостингом. '}, {'starts': '2011-11-01', 'ends': '2013-08-01', 'employer': 'МКК ФинансIT', 'city': 'Сергиев Посад', 'position': 'Инженер-программист', 'description': '• Анализ алгоритмов в среде Matlab и C; • Сетевые настройки в ОС Windows и Fedora; • Настройка VMware на ОС Windows и Fedora; • Поддержка работоспособности компьютеров.'}] ['Русский', 'Английский'] [{'year': 2015, 'organization': 'Санкт-Петербургский государственный электротехнический университет "ЛЭТИ" им. В.И. Ульянова (Ленина), Санкт-Петербург', 'faculty': 'Факультет компьютерных технологий и информатики', 'specialty': 'УТС', 'result': '', 'education_type': 'Основное', 'education_level': 'Высшее'}, {'year': 2013, 'organization': 'Санкт-Петербургский государственный электротехнический университет "ЛЭТИ" им. В.И. Ульянова (Ленина), Санкт-Петербург', 'faculty': 'Факультет компьютерных технологий и информатики', 'specialty': 'Математическое обеспечение ЭВМ', 'result': '', 'education_type': 'Основное', 'education_level': 'Высшее'}, {'year': 2022, 'organization': "Jenny's School", 'faculty': '', 'specialty': "Jenny's School", 'result': 'Английский язык - intermediate', 'education_type': 'Сертификаты', 'education_level': None}]
"""

In [18]:
classifier(text)

[{'entity_group': 'first_name',
  'score': 0.9997799,
  'word': 'Пётр',
  'start': 0,
  'end': 4},
 {'entity_group': 'last_name',
  'score': 0.9972767,
  'word': 'Игнатьев',
  'start': 5,
  'end': 13},
 {'entity_group': 'birth_date',
  'score': 0.74735165,
  'word': '1992 - 09 - 10',
  'start': 14,
  'end': 24},
 {'entity_group': 'country',
  'score': 0.99985635,
  'word': 'Россия',
  'start': 25,
  'end': 31},
 {'entity_group': 'city',
  'score': 0.9996874,
  'word': 'Санкт - Петербург',
  'start': 32,
  'end': 47},
 {'entity_group': 'about',
  'score': 0.9932829,
  'word': 'None',
  'start': 48,
  'end': 52},
 {'entity_group': 'key_skills',
  'score': 0.99538875,
  'word': 'Java, Java SE, Spring Framework, Spring Boot, PostgreSQL, Kafka, Apache Maven, OpenShift, Hibernate ORM, Liquibase, Jenkins, Bitbucket, CSS, HTML, JDBC, jUnit, Spring Test, Mockito, Git, GitLab, Jira, Confluence',
  'start': 53,
  'end': 264},
 {'entity_group': 'experienceItem',
  'score': 0.99859464,
  'word': "[